<a href="https://colab.research.google.com/github/phmouras/Projeto_Pos/blob/main/tst_automatic_A0_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

N = 30                                                   # Truncation ordem

L0 = 5                                             # Map parameter 

col = np.cos(np.arange(2*N + 3)*math.pi /(2*N + 2))      # collocation points (Verificado)
                               
colr = col[1:N+2]                                                         # physical domain (Verificado)             

r1 = L0 * colr/(np.sqrt(1-colr**2)) 
r = np.flip(r1)                                           # physical domain (Verificado)  

#out_r = open('r_30_L02.txt', 'a')
#out_r.write(' ' +' '.join(str('%.18f'%n) for n in r)+'\n')
#out_r.close()


# Base Matrix (Tchebyshev Polinomials): 

SB = np.zeros([N+2,N+1])
rSB = np.zeros([N+2,N+1])
rrSB = np.zeros([N+2,N+1])


for i in range(N+1+1):                                    # SB[:,0] = function in origin
  SB[i,] = np.sin((2*i+1)*np.arctan(L0/r))                                                  

for i in range(N+1+1):
  rSB[i,] = -np.cos((2*i+1)*np.arctan(L0/r))*(2*i+1)*L0/(r**2*(1+L0**2/r**2)) 

for i in range(N+1+1):
  rrSB[i,] = -np.sin((2*i+1)*np.arctan(L0/r))*(2*i+1)**2*L0**2/(r**4*(1+L0**2/r**2)**2)+2*np.cos((2*i+1)*np.arctan(L0/r))*(2*i+1)*L0/(r**3*(1+L0**2/r**2))-2*np.cos((2*i+1)*np.arctan(L0/r))*(2*i+1)*L0**3/(r**5*(1+L0**2/r**2)**2)                     

# Base function



psi = SB[0:N+1,:]        # origin included in psi[:,0]
rpsi = rSB[0:N+1,:]
rrpsi = rrSB[0:N+1,:]



# Initial conditions of Phi (Scalar field)                                    

r0 = 0

sigma = 1

Alpha_origin = []
phi_origin = []
L2HC = []
phi_set = []

# Equation for Krr: momentum constraint

# Initial values of Krr: Base functions of Krr

# In r = 0: 

SB1 = 1/2*(SB[1:(N+1),:] + SB[0:(N),:])          # VERIFICADO
rSB1 = 1/2*(rSB[1:(N+1),:] + rSB[0:(N),:])
rrSB1 = 1/2*(rrSB[1:(N+1),:] + rrSB[0:(N),:])

# Base functions for Beta

SB2 = np.zeros([N,N+1])      
rSB2 = np.zeros([N,N+1])
rrSB2 = np.zeros([N,N+1])



for i in range(N):                                                   # VERIFICADO
  SB2[i,] = np.sin((2*(i+1/2)+1)*np.arctan(L0/r))                                                  


for i in range(N):
  rSB2[i,] = -np.cos((2*i+2)*np.arctan(L0/r))*(2*i+2)*L0/(r**2*(1+L0**2/r**2))


for i in range(N):
  rrSB2[i,] = -np.sin((2*i+2)*np.arctan(L0/r))*(2*i+2)**2*L0**2/(r**4*(1+L0**2/r**2)**2)+2*np.cos((2*i+2)*np.arctan(L0/r))*(2*i+2)*L0/(r**3*(1+L0**2/r**2))-2*np.cos((2*i+2)*np.arctan(L0/r))*(2*i+2)*L0**3/(r**5*(1+L0**2/r**2)**2)

Nq = int(3/2*N)           # Quadrature truncation

gauss_quadrature = np.polynomial.legendre.leggauss(Nq + 1) 

new_col = gauss_quadrature[0]            # Legendre quadrature points


# Legendre Polinomials

P = np.zeros([Nq+3,Nq+1])
colP = np.zeros([Nq+3,Nq+1])

P[0,] = 1
P[1,] = new_col

colP[0,] = 0
colP[1,] = 1

for i in range(2,Nq+3):
  P[i,] = ((2*i-1)*new_col*P[i-1,] - (i-1)*P[i-2,])/(i)   

for i in range(2,Nq+3):
  colP[i,] = i*P[i-1] + new_col*colP[i-1]

P_max = P[Nq+1]

colP_max = colP[Nq+1]

wq_col = 2/((1-new_col**2)*colP_max**2)    # Legendre weight (Verificado)

rq = L0*(1+new_col)/(1-new_col)            # Physical quadrature domain

qSB = np.zeros([Nq+1,Nq+1])
qrSB = np.zeros([Nq+1,Nq+1])
qrrSB = np.zeros([Nq+1,Nq+1])


for i in range(Nq+1):
  qSB[i,] = np.sin((2*i+1)*np.arctan(L0/rq))                                                  

for i in range(Nq+1):
  qrSB[i,] = -np.cos((2*i+1)*np.arctan(L0/rq))*(2*i+1)*L0/(rq**2*+L0**2)

for i in range(Nq+1):
  qrrSB[i,] = -np.sin((2*i+1)*np.arctan(L0/rq))*(2*i+1)**2*L0**2/(rq**4*(1+L0**2/rq**2)**2)+2*np.cos((2*i+1)*np.arctan(L0/rq))*(2*i+1)*L0/(rq**3*(1+L0**2/rq**2))-2*np.cos((2*i+1)*np.arctan(L0/rq))*(2*i+1)*L0**3/(rq**5*(1+L0**2/rq**2)**2)                     

qpsi = qSB[0:N+1,:]
rqpsi = qrSB[0:N+1,:]
rrqpsi = qrrSB[0:N+1,:]


qSB1 = 1/2*(qSB[1:(N+1),:] + qSB[0:(N),:])          # VERIFICADO
rqSB1 = 1/2*(qrSB[1:(N+1),:] + qrSB[0:(N),:])
rrqSB1 = 1/2*(qSB[1:(N+1),:] + qrrSB[0:(N),:])



#A0 = 0.95 # . -> disperse 0.9 ;  # . 0.95 -> colapse       

A0_0 = np.linspace(0.9, 0.94) 

A0 = 0.94

while t<tf:



  Phi_init = A0*r**2*(np.exp(-(r-r0)**2/sigma**2) + np.exp(-(r+r0)**2/sigma**2))          # Phi initial data out of origin (Verificado)

  inv_psi = np.linalg.inv(psi)
    
  a0 = np.dot(Phi_init, inv_psi)  # coeficients a(0)  (Verificado)

  #out_a = open('a0_30_L02.txt', 'a')
  #out_a.write(' ' +' '.join(str('%.18f'%n) for n in a0)+'\n')
  #out_a.close()


  Phi = np.dot(a0, psi)        # approximative solution in t = 0
  rPhi= np.dot(a0, rpsi)

  ########################### Plot: Initial Conditions of Phi

  M = 3000       # plot truncation

  rplot = np.linspace(0.00000000000000001,15,M)

  colplot = rplot/np.sqrt(L0**2 + rplot**2)

  SBplot = np.zeros([N+1,M])
  rSBplot = np.zeros([N+1,M])
  rrSBplot = np.zeros([N+1,M])

  for i in range(N+1):
    SBplot[i,] = np.sin((2*i+1)*np.arctan(L0/rplot))

  for i in range(N+1):
    rSBplot[i,] = -np.cos((2*i+1)*np.arctan(L0/rplot))*(2*i+1)*L0/(rplot**2*(1+L0**2/rplot**2)) 

  for i in range(N+1):
    rrSBplot[i,] = -np.sin((2*i+1)*np.arctan(L0/rplot))*(2*i+1)**2*L0**2/(rplot**4*(1+L0**2/rplot**2)**2)+2*np.cos((2*i+1)*np.arctan(L0/rplot))*(2*i+1)*L0/(rplot**3*(1+L0**2/rplot**2))-2*np.cos((2*i+1)*np.arctan(L0/rplot))*(2*i+1)*L0**3/(rplot**5*(1+L0**2/rplot**2)**2)                     

  psiplot = SBplot[0:(N+1),:] 
  rpsiplot = rSBplot[0:(N+1),:]
  rrpsiplot = rrSBplot[0:(N+1),:]

  Phiplot_init = A0*rplot**2*(np.exp(-(rplot-r0)**2/sigma**2)+np.exp(-(rplot+r0)**2/sigma**2))     

  #Phiplot_init = np.hstack((0, Phiplot_int))

  Phiplot = np.dot(a0, psiplot)

  plt.plot(rplot, Phiplot, rplot, Phiplot_init, "--r")   #(Verificado)
  plt.xlabel('r')
  plt.xlim(0,8)
  plt.show()

  Pi_0 = np.zeros(N+1)
  b0 = np.dot(Pi_0, psi)
  Pi = np.dot(b0, psi)

  c0 = np.zeros([N+1])     # guess value
  for i in range(N+1):
    c0[i] = 0.0001

  coeforigin = 1/L0**2*(-(-1)**(np.arange(N+1))*(2*np.arange(1,N+2)-1)**2)    # 1/r * d\phi/dr

  tol = 1e-19
  #cnew = 0 
  #err = abs(c0 - cnew)

  n = 0
  nf = 100

  # Newton Raphson loop
  #while max(err) >= tol:
  while n <= nf:
    rChi = np.dot(c0 ,rpsi)
    rrChi = np.dot(c0, rrpsi)
    H0_0 = 4*rrChi[0] + 8*np.dot(coeforigin,c0) + 1/2*(rPhi[0])**2
    H0_int = 4*rChi[1:N+1]**2 + 4*rrChi[1:N+1] + 8/r[1:N+1]*rChi[1:N+1] + 1/2*rPhi[1:N+1]**2
    H0 = np.hstack((H0_0, H0_int))
    J1 = rChi*rpsi
    JH_0 = 8*J1[:,0]+ 4*rrpsi[:,0] + 8*coeforigin
    JH_int = 8*J1[:,1:N+1] + 4*rrpsi[:,1:N+1] + 8/r[1:N+1]*rpsi[:,1:N+1]  
    JH = np.column_stack((JH_0, JH_int))
    inv_JH = np.linalg.inv(JH)
    cnew = c0
    c0 = c0 - np.dot(H0, inv_JH)
    err = abs(c0 - cnew)
    print(max(err))
    if  max(err) <= tol:
      print(c0)
      break
    n = n + 1
  Chi = np.dot(c0, psi)

  # Runge Kutta 4th order

  h = 0.001   # step size
  tf = 9

  It = int(tf/h)

  #t1 = np.linspace(0, tf, It) 

  t = 0


  V = 0

  out_a = open('Alpha_origin.txt', 'a')
  out_a.truncate(0)


  out_p = open('phi_origin.txt', 'a')
  out_p.truncate(0)

  #matriz = np.concatenate((array1, array2))

  while t <= tf:
    
    # First step
    Phi = np.dot(a0, psi)      
    rPhi = np.dot(a0, rpsi)
    rrPhi = np.dot(a0, rrpsi)      
    Pi = np.dot(b0, psi) 
    rPi= np.dot(b0, rpsi)
    Chi = np.dot(c0, psi) 
    rChi = np.dot(c0, rpsi)
    rrChi = np.dot(c0, rrpsi)   
    Matrix_Krr = 2*rChi[1:N+1]*SB1[:,1:N+1] + rSB1[:,1:N+1] + 3/r[1:N+1]*SB1[:,1:N+1]   # (p, p)
    inv_matrix_krr = np.linalg.inv(Matrix_Krr)
    rhsk = - Pi[1:N+1]*rPhi[1:N+1]*np.exp(4*Chi[1:N+1]) # (p,)
    ck0 = np.dot(rhsk, inv_matrix_krr)  # (p,)
    Krr = np.dot(ck0, SB1)              # (p+1,)     
    rKrr = np.dot(ck0, rSB1)
    Matrix_Alpha_0 = rrpsi[:,0] + 2*coeforigin - np.exp(4*Chi[0])*Pi[0]**2*psi[:,0]
    Matrix_Alpha_int = rrpsi[:,1:N+1] + 2*(1/r[1:N+1] + rChi[1:N+1])*rpsi[:,1:N+1] - 3/2*np.exp(-4*Chi[1:N+1])*Krr[1:N+1]**2*psi[:,1:N+1] - np.exp(4*Chi[1:N+1])*(Pi[1:N+1]**2 - V)*psi[:,1:N+1] 
    Matrix_Alpha = np.column_stack((Matrix_Alpha_0, Matrix_Alpha_int))
    inv_matrix_alpha = np.linalg.inv(Matrix_Alpha)
    rhsal = 3/2*np.exp(-4*Chi)*Krr**2 + np.exp(4*Chi)*(Pi**2-V)
    al0 = np.dot(rhsal, inv_matrix_alpha)
    Alpha = 1 + np.dot(al0, psi)
    rAlpha = np.dot(al0, rpsi)
    rrAlpha = np.dot(al0, rrpsi)
    Matrix_Beta = rSB2[:,1:N+1] - SB2[:,1:N+1]/r[1:N+1]
    inv_matrix_beta = np.linalg.inv(Matrix_Beta)
    rhsbe = 3/2*Alpha[1:N+1]*np.exp(-4*Chi[1:N+1])*Krr[1:N+1]
    be0 = np.dot(rhsbe , inv_matrix_beta)
    Beta = np.dot(be0, SB2)
    rBeta = np.dot(be0, rSB2)
    db_origin = 2*np.exp(-4*Chi[0])*Alpha[0]*np.dot(a0, coeforigin) + np.exp(-4*Chi[0])*Alpha[0]*rrPhi[0]
    db_int = Beta[1:N+1]*rPi[1:N+1] + np.exp(-4*Chi[1:N+1])*(2*Alpha[1:N+1]/r[1:N+1] + rAlpha[1:N+1] + 2*rChi[1:N+1]*Alpha[1:N+1])*rPhi[1:N+1] + np.exp(-4*Chi[1:N+1])*Alpha[1:N+1]*rrPhi[1:N+1] - Alpha[1:N+1]* V
    db = np.dot(np.hstack((db_origin, db_int)), inv_psi)
    dc = np.dot(Beta*rChi + rBeta/2 - Alpha/2*np.exp(-4*Chi)*Krr, inv_psi)   
    da = np.dot(Alpha*Pi + Beta*rPhi, inv_psi)   
    K1 = h*(dc)
    L1 = h*(da)
    N1 = h*(db)
    
    # L2-error associated to the Hamiltonian constraint
    qPhi = np.dot(a0, qpsi)      
    rqPhi= np.dot(a0, rqpsi)
    qPi = np.dot(b0, qpsi) 
    qChi = np.dot(c0, qpsi)
    rqChi = np.dot(c0, rqpsi)
    rrqChi = np.dot(c0, rrqpsi) 
    qKrr = np.dot(ck0, qSB1)
    H = 4*rqChi**2 + 4*rrqChi + 8*rqChi/rq + 3/4*np.exp(-4*qChi)*qKrr**2 + np.exp(4*qChi)*(1/2*qPi**2 + np.exp(-4*qChi)/2*rqPhi**2)   # Hamiltonian constraint (HC)
    L2HC.append((1/2*np.dot(H**2,wq_col))**1/2)    # L2 error of HC 
    
    # Alpha origin
    Alpha_0 = 1 + np.dot(al0, psi[:,0])
    Alpha_origin.append(Alpha_0)  # = Alphacenter in matlab
    out_a.write(str(Alpha_0))
    out_a.write(',\n')
    #print(Alpha_origin)
    
    # Phi origin:  
    phi_0 = np.dot(a0, psi[:,0])
    phi_origin.append(phi_0)
    out_p.write(str(phi_0))
    out_p.write(',\n')
  #  print(phi_origin)
      
    # Second step
    Phi = np.dot(a0 + L1/2, psi)     
    rPhi= np.dot(a0 + L1/2 , rpsi)
    rrPhi = np.dot(a0 + L1/2, rrpsi) 
    Pi = np.dot(b0 + N1/2, psi)
    rPi= np.dot(b0 + N1/2, rpsi)
    Chi = np.dot(c0 + K1/2, psi)
    rChi = np.dot(c0 + K1/2, rpsi)
    rrChi = np.dot(c0 + K1/2, rrpsi)
    Matrix_Krr = 2*rChi[1:N+1]*SB1[:,1:N+1] + rSB1[:,1:N+1] + 3/r[1:N+1]*SB1[:,1:N+1]   # (p, p)
    inv_matrix_krr = np.linalg.inv(Matrix_Krr)
    rhsk = - Pi[1:N+1]*rPhi[1:N+1]*np.exp(4*Chi[1:N+1]) # (p,)
    ck0 = np.dot(rhsk, inv_matrix_krr)  # (p,)
    Krr = np.dot(ck0, SB1)              # (p+1,)     
    rKrr = np.dot(ck0, rSB1)
    Matrix_Alpha_0 = rrpsi[:,0] + 2*coeforigin - np.exp(4*Chi[0])*Pi[0]**2*psi[:,0]
    Matrix_Alpha_int = rrpsi[:,1:N+1] + 2*(1/r[1:N+1] + rChi[1:N+1])*rpsi[:,1:N+1] - 3/2*np.exp(-4*Chi[1:N+1])*Krr[1:N+1]**2*psi[:,1:N+1] - np.exp(4*Chi[1:N+1])*(Pi[1:N+1]**2 - V)*psi[:,1:N+1] 
    Matrix_Alpha = np.column_stack((Matrix_Alpha_0, Matrix_Alpha_int))
    inv_matrix_alpha = np.linalg.inv(Matrix_Alpha)
    rhsal = 3/2*np.exp(-4*Chi)*Krr**2 + np.exp(4*Chi)*(Pi**2-V)
    al0 = np.dot(rhsal, inv_matrix_alpha)
    Alpha = 1 + np.dot(al0, psi)
    rAlpha = np.dot(al0, rpsi)
    rrAlpha = np.dot(al0, rrpsi)
    Matrix_Beta = rSB2[:,1:N+1] - SB2[:,1:N+1]/r[1:N+1]
    inv_matrix_beta = np.linalg.inv(Matrix_Beta)
    rhsbe = 3/2*Alpha[1:N+1]*np.exp(-4*Chi[1:N+1])*Krr[1:N+1]
    be0 = np.dot(rhsbe , inv_matrix_beta)
    Beta = np.dot(be0, SB2)
    rBeta = np.dot(be0, rSB2)
    db_origin = 2*np.exp(-4*Chi[0])*Alpha[0]*np.dot(a0, coeforigin) + np.exp(-4*Chi[0])*Alpha[0]*rrPhi[0]
    db_int = Beta[1:N+1]*rPi[1:N+1] + np.exp(-4*Chi[1:N+1])*(2*Alpha[1:N+1]/r[1:N+1] + rAlpha[1:N+1] + 2*rChi[1:N+1]*Alpha[1:N+1])*rPhi[1:N+1] + np.exp(-4*Chi[1:N+1])*Alpha[1:N+1]*rrPhi[1:N+1] - Alpha[1:N+1]* V
    db = np.dot(np.hstack((db_origin, db_int)), inv_psi)
    dc = np.dot(Beta*rChi + rBeta/2 - Alpha/2*np.exp(-4*Chi)*Krr, inv_psi)   
    da = np.dot(Alpha*Pi + Beta*rPhi, inv_psi)   
    K2 = h*(dc)
    L2 = h*(da)
    N2 = h*(db)

    # Third step
    Phi = np.dot(a0 + L2/2, psi)     
    rPhi = np.dot(a0 + L2/2 , rpsi)
    rrPhi = np.dot(a0 + L2/2, rrpsi) 
    Pi = np.dot(b0 + N2/2, psi)
    rPi= np.dot(b0 + N2/2, rpsi)
    Chi = np.dot(c0 + K2/2, psi)
    rChi = np.dot(c0 + K2/2, rpsi)
    rrChi = np.dot(c0 + K2/2, rrpsi)
    Matrix_Krr = 2*rChi[1:N+1]*SB1[:,1:N+1] + rSB1[:,1:N+1] + 3/r[1:N+1]*SB1[:,1:N+1]   # (p, p)
    inv_matrix_krr = np.linalg.inv(Matrix_Krr)
    rhsk = - Pi[1:N+1]*rPhi[1:N+1]*np.exp(4*Chi[1:N+1]) # (p,)
    ck0 = np.dot(rhsk, inv_matrix_krr)  # (p,)
    Krr = np.dot(ck0, SB1)              # (p+1,)     
    rKrr = np.dot(ck0, rSB1)
    Matrix_Alpha_0 = rrpsi[:,0] + 2*coeforigin - np.exp(4*Chi[0])*Pi[0]**2*psi[:,0]
    Matrix_Alpha_int = rrpsi[:,1:N+1] + 2*(1/r[1:N+1] + rChi[1:N+1])*rpsi[:,1:N+1] - 3/2*np.exp(-4*Chi[1:N+1])*Krr[1:N+1]**2*psi[:,1:N+1] - np.exp(4*Chi[1:N+1])*(Pi[1:N+1]**2 - V)*psi[:,1:N+1] 
    Matrix_Alpha = np.column_stack((Matrix_Alpha_0, Matrix_Alpha_int))
    inv_matrix_alpha = np.linalg.inv(Matrix_Alpha)
    rhsal = 3/2*np.exp(-4*Chi)*Krr**2 + np.exp(4*Chi)*(Pi**2-V)
    al0 = np.dot(rhsal, inv_matrix_alpha)
    Alpha = 1 + np.dot(al0, psi)
    rAlpha = np.dot(al0, rpsi)
    rrAlpha = np.dot(al0, rrpsi)
    Matrix_Beta = rSB2[:,1:N+1] - SB2[:,1:N+1]/r[1:N+1]
    inv_matrix_beta = np.linalg.inv(Matrix_Beta)
    rhsbe = 3/2*Alpha[1:N+1]*np.exp(-4*Chi[1:N+1])*Krr[1:N+1]
    be0 = np.dot(rhsbe , inv_matrix_beta)
    Beta = np.dot(be0, SB2)
    rBeta = np.dot(be0, rSB2)
    db_origin = 2*np.exp(-4*Chi[0])*Alpha[0]*np.dot(a0, coeforigin) + np.exp(-4*Chi[0])*Alpha[0]*rrPhi[0]
    db_int = Beta[1:N+1]*rPi[1:N+1] + np.exp(-4*Chi[1:N+1])*(2*Alpha[1:N+1]/r[1:N+1] + rAlpha[1:N+1] + 2*rChi[1:N+1]*Alpha[1:N+1])*rPhi[1:N+1] + np.exp(-4*Chi[1:N+1])*Alpha[1:N+1]*rrPhi[1:N+1] - Alpha[1:N+1]* V
    db = np.dot(np.hstack((db_origin, db_int)), inv_psi)
    dc = np.dot(Beta*rChi + rBeta/2 - Alpha/2*np.exp(-4*Chi)*Krr, inv_psi)   
    da = np.dot(Alpha*Pi + Beta*rPhi, inv_psi)   
    K3 = h*(dc)
    L3 = h*(da)
    N3 = h*(db) 

    # Forth step
    Phi = np.dot(a0 + L3, psi)     
    rPhi= np.dot(a0 + L3 , rpsi)
    rrPhi = np.dot(a0 + L3, rrpsi) 
    Pi = np.dot(b0 + N3, psi)
    rPi= np.dot(b0 + N3, rpsi)
    Chi = np.dot(c0 + K3, psi)  
    rChi = np.dot(c0 + K3, rpsi)
    rrChi = np.dot(c0 + K3, rrpsi)
    Matrix_Krr = 2*rChi[1:N+1]*SB1[:,1:N+1] + rSB1[:,1:N+1] + 3/r[1:N+1]*SB1[:,1:N+1]   # (p, p)
    inv_matrix_krr = np.linalg.inv(Matrix_Krr)
    rhsk = - Pi[1:N+1]*rPhi[1:N+1]*np.exp(4*Chi[1:N+1]) # (p,)
    ck0 = np.dot(rhsk, inv_matrix_krr)  # (p,)
    Krr = np.dot(ck0, SB1)              # (p+1,)     
    rKrr = np.dot(ck0, rSB1)
    Matrix_Alpha_0 = rrpsi[:,0] + 2*coeforigin - np.exp(4*Chi[0])*Pi[0]**2*psi[:,0]
    Matrix_Alpha_int = rrpsi[:,1:N+1] + 2*(1/r[1:N+1] + rChi[1:N+1])*rpsi[:,1:N+1] - 3/2*np.exp(-4*Chi[1:N+1])*Krr[1:N+1]**2*psi[:,1:N+1] - np.exp(4*Chi[1:N+1])*(Pi[1:N+1]**2 - V)*psi[:,1:N+1] 
    Matrix_Alpha = np.column_stack((Matrix_Alpha_0, Matrix_Alpha_int))
    inv_matrix_alpha = np.linalg.inv(Matrix_Alpha)
    rhsal = 3/2*np.exp(-4*Chi)*Krr**2 + np.exp(4*Chi)*(Pi**2-V)
    al0 = np.dot(rhsal, inv_matrix_alpha)
    Alpha = 1 + np.dot(al0, psi)
    rAlpha = np.dot(al0, rpsi)
    rrAlpha = np.dot(al0, rrpsi)
    Matrix_Beta = rSB2[:,1:N+1] - SB2[:,1:N+1]/r[1:N+1]
    inv_matrix_beta = np.linalg.inv(Matrix_Beta)
    rhsbe = 3/2*Alpha[1:N+1]*np.exp(-4*Chi[1:N+1])*Krr[1:N+1]
    be0 = np.dot(rhsbe , inv_matrix_beta)
    Beta = np.dot(be0, SB2)
    rBeta = np.dot(be0, rSB2)
    db_origin = 2*np.exp(-4*Chi[0])*Alpha[0]*np.dot(a0, coeforigin) + np.exp(-4*Chi[0])*Alpha[0]*rrPhi[0]
    db_int = Beta[1:N+1]*rPi[1:N+1] + np.exp(-4*Chi[1:N+1])*(2*Alpha[1:N+1]/r[1:N+1] + rAlpha[1:N+1] + 2*rChi[1:N+1]*Alpha[1:N+1])*rPhi[1:N+1] + np.exp(-4*Chi[1:N+1])*Alpha[1:N+1]*rrPhi[1:N+1] - Alpha[1:N+1]* V
    db = np.dot(np.hstack((db_origin, db_int)), inv_psi)
    dc = np.dot(Beta*rChi + rBeta/2 - Alpha/2*np.exp(-4*Chi)*Krr, inv_psi)   
    da = np.dot(Alpha*Pi + Beta*rPhi, inv_psi)   
    K4 = h*(dc)
    L4 = h*(da)
    N4 = h*(db)  
    
    t = t + h
  #print(t) 
  #  if t > 9:
  #    h=0.0001

      
    # Evolution functions
    a0 = a0 + 1/6 * (L1 + 2*L2 + 2*L3 + L4)
    b0 = b0 + 1/6 * (N1 + 2*N2 + 2*N3 + N4)
    c0 = c0 + 1/6 * (K1 + 2*K2 + 2*K3 + K4)  
    phi_set.append(np.dot(a0, psiplot))

  #  pi_set[i,:] = np.dot(b0, psiplot)
  #  chi_set[i,:] = np.dot(c0, psiplot)

  if Alpha_origin < 0.05:
    A0 = A0_0[int(len(A0_0)/2)]
    print('colapsa')
    break

  if Alpha_origin > 0.05:
    A0 = 
  out_a.close()

  out_p.close()
              
  #out_b = open('phi_origin.txt', 'b')
  #out_b.write(' ' +' '.join(str('%.18f'%n) for n in phi_origin)+'\n')
  #out_b.close()  


  
t1 = np.linspace(0, tf, len(Alpha_origin)) 


In [ ]:
A0_0 = np.linspace(0.9, 0.95) 
 
A0 = A0_0[int(len(A0_0)/2)]
A0

0.9255102040816326

In [ ]:
for i in range(It):
  plt.plot(rplot, phi_set[i])

In [ ]:
Alpha_origin[8999]

In [ ]:
# Searching for critical amplitude:

plt.plot(t1, Alpha_origin, color = "g", label = "$A_0$ = {:}".format(A0))
plt.title("Alpha na origem para L0 = 5, N = {:}".format(N))
plt.ylabel(r"$\alpha(t,0)$")
plt.xlabel("t")
plt.legend()

In [ ]:
plt.plot(t1, phi_origin, color = "b", label = " = {:}".format(A0))
plt.title("Phi na origem para L0 = 5 e N = {:}".format(N) )
plt.ylabel("$\phi$(t,0)")
plt.xlabel("t")  
#plt.xlim(7.2,8.2)
plt.legend() 

In [ ]:
# Erro L2 of Hamiltonian constraint

plt.plot(t1,L2HC)
plt.yscale("log")
plt.ylabel("log(L2HC)")
plt.xlabel("t")
plt.title("log(L2HC) para $N = 600$, $L_0 = 2$ e A_0 = {:}".format(A0))